In [5]:
import pandas as pd
#pd.set_option('display.max_rows', None)

In [6]:
welcome_message_data = pd.read_csv(r"C:\Users\Ratan Kumar Jha\Desktop\welcome msg\ANALYSIS OUTPUT\FINAL_DATA\FINAL_DATA_ALL_MERGE_MISSING_DAYS_COUNT.csv")
welcome_message_data.drop(['Unnamed: 0'],axis=1, inplace=True)
pd.set_option('display.max_columns', None)

In [7]:
welcome_message_data.sample(1)

,Consumer ID,installation_number,New Meter No,Meter Replacement,Aging_Creation_Date,Detailed_Remarks,Utility Name,Replacement Date,Aging,Bucket,Bucket1,Category Name,Aging_Creation_Date_BACKDATE,Year-Week,Week,Month,Agging_bucket_30,Resolved Cases,Count,lead_status,lead_date,meter_aging
1236868,5004153649,5004153649,LT8007292,NaN,2024-07-18,Work Order Not Closed,SBPD,2024-07-16,2,0 - 03,02 - 03,NDS-IID(B),2024-07-14,2024-28,29,7,<=30,0,1,Work Order Not Closed,2024-07-19,1


In [8]:
welcome_message_data.columns

Index(['Consumer ID', 'installation_number', 'New Meter No',
       'Meter Replacement', 'Aging_Creation_Date', 'Detailed_Remarks',
       'Utility Name', 'Replacement Date', 'Aging', 'Bucket', 'Bucket1',
       'Category Name', 'Aging_Creation_Date_BACKDATE', 'Year-Week', 'Week',
       'Month', 'Agging_bucket_30', 'Resolved Cases', 'Count', 'lead_status',
       'lead_date', 'meter_aging'],
      dtype='object')

In [9]:
welcome_message_data.drop(['Consumer ID',
        'Meter Replacement',
        'Utility Name', 'Replacement Date', 
        'Bucket',
       'Bucket1', 'Category Name', 
       'Aging_Creation_Date_BACKDATE', 'Year-Week', 'Week',
       'Month','Agging_bucket_30','Resolved Cases','Count','lead_status','lead_date','Aging'],axis=1, inplace=True)


# this will sort the meter and aging_Creation_date
welcome_message_data = welcome_message_data.sort_values(['New Meter No', 'Aging_Creation_Date'], ascending=[True, True])

In [10]:
# after partition or group by we did lead of detailed remark

welcome_message_data['lead_Detailed_remark'] = welcome_message_data.groupby('New Meter No')['Detailed_Remarks'].shift(-1)

In [12]:
welcome_message_data.head(3)

,installation_number,New Meter No,Aging_Creation_Date,Detailed_Remarks,meter_aging,lead_Detailed_remark
0,5004017369,GL8548353,2024-02-23,Work Order Not Closed,1,Installed in MDM but CA not Assigned from DISCOM
1,5004017369,GL8548353,2024-02-24,Installed in MDM but CA not Assigned from DISCOM,2,"NMI Success, AMR Not ""Y"", MR Success and DP Re..."
2,5004017369,GL8548353,2024-02-26,"NMI Success, AMR Not ""Y"", MR Success and DP Re...",1,"NMI Success, AMR Not ""Y"", MR Success and DP Re..."


In [13]:
import pandas as pd

output_list = []

for meter, group in welcome_message_data.groupby('New Meter No'):
    sum_meter_aging = 0  

    for i in range(len(group)):
        starting_remark = group['Detailed_Remarks'].iloc[i]
        lead_remark = group['lead_Detailed_remark'].iloc[i]
        meter_aging_value = group['meter_aging'].iloc[i]

        if starting_remark != lead_remark:
            sum_meter_aging += meter_aging_value  
            date = group['Aging_Creation_Date'].iloc[i]  
            output_list.append({
                'meter_no': meter,
                'date': date,
                'Detailed_Remark': starting_remark,
                'Sum_meter_aging': sum_meter_aging
            })
        
            sum_meter_aging = 0
        else:
          
            sum_meter_aging += meter_aging_value  
           

# Convert to DataFrame
output_df = pd.DataFrame(output_list)

# Print the DataFrame
#print(output_df)


In [14]:
output_df.head()

,meter_no,date,Detailed_Remark,Sum_meter_aging
0,GL8548353,2024-02-23,Work Order Not Closed,1
1,GL8548353,2024-02-24,Installed in MDM but CA not Assigned from DISCOM,2
2,GL8548353,2024-02-28,"NMI Success, AMR Not ""Y"", MR Success and DP Re...",3
3,GL8548353,2024-02-29,"NMI Success, AMR Not ""Y"", MR Success and DP No...",1
4,GL8548353,2024-03-02,"Communicating not ""Y"" in MDM",2


In [15]:
output_df.to_csv(r"C:\Users\Ratan Kumar Jha\Desktop\welcome msg\ANALYSIS OUTPUT\FINAL_DATA\output_df.csv")

In [18]:
output_df.shape

(419478, 4)

In [19]:
import pandas as pd


last_rows = output_df.groupby('meter_no').last().reset_index()

last_rows.to_csv(r"C:\Users\Ratan Kumar Jha\Desktop\welcome msg\ANALYSIS OUTPUT\FINAL_DATA\last_rows.csv")


In [20]:
last_rows.head(2)

,meter_no,date,Detailed_Remark,Sum_meter_aging
0,GL8548353,2024-03-02,"Communicating not ""Y"" in MDM",2
1,GL8548461,2024-04-03,"Communicating not ""Y"" in MDM",1


In [16]:
output_df[output_df['meter_no']=='GL8548474']

,meter_no,date,Detailed_Remark,Sum_meter_aging
8,GL8548474,2024-01-05,Never Communicating Meter (O&M-IT),4
